<a href="https://www.kaggle.com/code/erandas/identifying-if-it-s-a-venomous-snake?scriptVersionId=141285618" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Goal of this model is to identify if it is a Scarlet Kingsnake (non-venemous) or if it is a Coral Snake(venomous).

I will use duckduckgo search engine images to accomplish this task.

In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search

## Download Images of Scarlet Kingsnakes

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images( term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
urls = search_images('Scarlet Kingsnake photos', max_images=1)
urls[0]

In [ ]:
from fastdownload import download_url
dest = 'non-venomous.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

## Download Images Coral snakes


In [ ]:
download_url(search_images('Coral snake photos', max_images=1)[0], 'venomous.jpg', show_progress=False)
Image.open('venomous.jpg').to_thumb(256,256)

In [ ]:
searches = 'Scarlet Kingsnake','Coral snake'
path = Path('venmous_or_not')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents =True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)
    
    resize_images(path/o, max_size=400, dest=path/o)

## Training our model


In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=10)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

## Using this model to identify an actual image of a Coral snake:


In [ ]:
is_venomous,_,probs = learn.predict(PILImage.create('venomous.jpg'))
if is_venomous:
    print("This is a Coral snake: Poisonous!.")
    print(f"Probability it's a venomous snake is:{probs[0]:.4f}")
else:
    print("This is not a Coral Snake, Probably a Scarlet Kingsnake: Not - Poisonous.")
    print(f"Probability it's non-venomous snake is:{1 - probs[0]:.4f}")
    



